In [19]:
import copy
import pandas as pd
import random

In [20]:
class Player:
    """This is a player with his/her cards and points."""
    def __init__(self, name, st_give, st_take):
        self.name = name
        self.st_give = st_give
        self.st_take = st_take
        self.ochki = 0
        self.cards = []
        
    def show_info(self):
        print(f'Name: {self.name}\nStrategies: {self.st_give}, {self.st_take}\nCards: {self.cards}\nPoints: {self.ochki}')
         
    def give_card(self):
        """This is how the player makes a move."""
        return self.cards.pop()
    
    def zagrabastat(self, ochki):
        """This method gives points to player."""
        self.ochki += ochki
        
    def take_cards(self, cards):
        """This method gives cards to player."""
        self.cards = cards
        
    

In [21]:
class Deck:
    def __init__(self):
        self.cards = list(range(1, 105))
        random.shuffle(self.cards)
        self.ochki = self.get_points()
    
    def get_points(self):
        result = {}
        for card in self.cards:
            if card == 55:
                points = 7
            elif card % 10 == 0:
                points = 3
            elif card % 5 == 0:
                points = 2
            elif card % 11 == 0:
                points = 5
            else:
                points = 1
            result[card] = points
        return result
    
    def deal_cards(self, count):
        if count <1:
            print('Enter a positive number')
        elif count == 1:
            return self.cards.pop()
        elif count >1:
            return self.pop_cards(count)
        
    def pop_cards(self, count):
        result = []
        for card in range(count):
            result.append(self.cards.pop())
        return result
    
    def show_info(self, count=104):
        print('Showing DECK info')
        print(f"Cards left: {len(self.cards)}")
        if count == 104:
            print(f'Showing all cards: {self.cards[-count:]}')
        else:
            print(f'Showing last {count} cards: {self.cards[-count:]}')

In [22]:
class Rows:
    def __init__(self, cards, ochki):
        self.ochki = ochki
        empty_dict = {"cards": [], "count": 0, "ochki": 0}
        self.data = {"r0": copy.deepcopy(empty_dict),
                     "r1": copy.deepcopy(empty_dict),
                     "r2": copy.deepcopy(empty_dict),
                     "r3": copy.deepcopy(empty_dict),}
        for key, value in self.data.items():
            # add a card to the row
            one_card = cards.pop()
            self.data[key]["cards"].append(one_card)
            # increment cards' count
            self.data[key]["count"] += 1
            # count points (ochki)
            self.data[key]["ochki"] += self.ochki[one_card]

    def show_info(self):
        print("\nShowing rows info:")
        for key, value in self.data.items():
            print(f"{key}: {value}")

    def game_round(self, players):
        # create dict card>>player (as object)
        cs2p = {}
        for p in players:
            card = p.give_card()
            cs2p[card] = p
        
        # copy cards to a list
        players_cards = [c for c in cs2p.keys()]
        
        # take minimal card and put to row
        for move in range(len(players)):
            # find minimal value
            min_value = min(players_cards)
            # find index of the card
            min_id = players_cards.index(min_value)
            # pop minimal card
            min_card = players_cards.pop(min_id)
            player_min = cs2p[min_card]
            self.put_card_on_row(min_card, player_min)
        self.show_info()
    
    def put_card_on_row(self, card, player):
        # calculate deltas
        deltas = {}
        for key, value in self.data.items():
            deltas[key] = card - value["cards"][-1]
        # print(deltas)
        
        # find row with minimal positive delta
        row_found = None
        mpd = 105  # minimum positive delta
        for key, value in deltas.items():
            if value > 0:
                if value < mpd:
                    mpd = value
                    row_found = key
       
        # Player is going to zagrabastat here or add card to the row
        if row_found:
            if self.data[row_found]["count"] == 5:
                #print(f"row {row_found} has five cards")
                # ZAGRABASTAT
                player.zagrabastat(self.data[row_found]["ochki"])
                # remove the cards from the row and make "card" the first and only card in the row
                self.data[row_found]["cards"] = [card]
                self.data[row_found]["count"] = 1
                self.data[row_found]["ochki"] = self.ochki[card]

                # pass "ochki" to the player
                
                
            else:
                #print(f"row {row_found} has less than five cards")
                # add the card to the row
                self.data[row_found]["cards"].append(card)
                # increment cards' count
                self.data[row_found]["count"] += 1
                # count points (ochki)
                self.data[row_found]["ochki"] += self.ochki[card]

        else:
            #print("Player grabastaet r0")
            # Player grabastaet the first row "r0"
            player.zagrabastat(self.data["r0"]["ochki"])
            self.data["r0"]["cards"] = [card]
            self.data["r0"]["count"] = 1
            self.data["r0"]["ochki"] = self.ochki[card]


# Prepare for game

In [23]:
def one_game():
    # create a deck
    deck = Deck()
    
    # create the rows
    rows = Rows(deck.deal_cards(4), deck.ochki)
    
    # creating players
    p0 = Player("Jenny", "give_random", "take_random")
    p1 = Player("Tonia", "give_random", "take_random")
    p2 = Player("Roma", "give_random", "take_random")
    p3 = Player("Nastia", "give_random", "take_random")
    players = [p0, p1, p2, p3]
    # give them cards
    for p in players:
        p.take_cards(deck.deal_cards(10))
    
    # simulate 10 moves (one_game)
    for round in range(10):
        rows.game_round(players)

    
    # rows.show_info()
    # for p in players:
    #     p.show_info()
    return [p.ochki for p in players]

# Games

In [24]:
result = {f"p{x}": [] for x in range(4)}
for game in range(1):
    ochki = one_game()
    for x in range(4): 
        result[f'p{x}'].append(ochki[x])


Showing rows info:
r0: {'cards': [76, 90], 'count': 2, 'ochki': 4}
r1: {'cards': [56], 'count': 1, 'ochki': 1}
r2: {'cards': [63], 'count': 1, 'ochki': 1}
r3: {'cards': [8, 20, 26, 27], 'count': 4, 'ochki': 6}

Showing rows info:
r0: {'cards': [76, 90, 96, 101], 'count': 4, 'ochki': 6}
r1: {'cards': [56], 'count': 1, 'ochki': 1}
r2: {'cards': [63, 78], 'count': 2, 'ochki': 2}
r3: {'cards': [8, 20, 26, 27, 48], 'count': 5, 'ochki': 7}

Showing rows info:
r0: {'cards': [5, 16, 40], 'count': 3, 'ochki': 6}
r1: {'cards': [56], 'count': 1, 'ochki': 1}
r2: {'cards': [63, 78, 100], 'count': 3, 'ochki': 5}
r3: {'cards': [8, 20, 26, 27, 48], 'count': 5, 'ochki': 7}

Showing rows info:
r0: {'cards': [28, 30], 'count': 2, 'ochki': 4}
r1: {'cards': [56, 94], 'count': 2, 'ochki': 2}
r2: {'cards': [63, 78, 100, 104], 'count': 4, 'ochki': 6}
r3: {'cards': [8, 20, 26, 27, 48], 'count': 5, 'ochki': 7}

Showing rows info:
r0: {'cards': [18, 35, 38, 47], 'count': 4, 'ochki': 5}
r1: {'cards': [56, 94], '

In [17]:
ochki

[22, 11, 15, 11]

In [18]:
df = pd.DataFrame(result)
df

,p0,p1,p2,p3
0,18,5,16,21
1,7,6,25,22
2,18,13,9,12
3,18,5,15,6
4,12,19,7,4
...,...,...,...,...
995,13,16,10,10
996,18,12,8,11
997,16,12,7,3
998,10,9,0,24


In [79]:
df[23:98]

,p0,p1,p2,p3
23,11,25,3,6
24,2,17,7,35
25,9,24,24,2
26,9,18,14,15
27,8,9,16,29
...,...,...,...,...
93,19,7,22,7
94,10,22,8,16
95,15,5,26,1
96,13,13,18,7


In [84]:
df["p0"].mean()

13.097873

In [26]:
rabbit.show_info()
sony.show_info()
cactus.show_info()

Name: Rabbit
Strategies: give_random, take_random
Cards: [50, 41, 5, 64, 10, 3, 74, 57, 78, 35]
Points: 0
Name: Sony
Strategies: give_random, take_random
Cards: [46, 91, 2, 54, 6, 56, 49, 22, 45, 86]
Points: 0
Name: Cactus
Strategies: give_random, take_random
Cards: [85, 19, 25, 103, 61, 97, 42, 88, 48, 30]
Points: 0


In [27]:
deck.show_info()

Showing DECK info
Cards left: 74
Showing all cards: [1, 71, 27, 33, 101, 24, 87, 51, 37, 44, 58, 76, 92, 65, 63, 72, 96, 17, 77, 70, 69, 14, 59, 7, 40, 12, 66, 82, 95, 32, 104, 99, 100, 67, 83, 23, 75, 53, 47, 15, 11, 81, 38, 13, 73, 94, 84, 26, 9, 36, 79, 102, 80, 62, 21, 20, 16, 98, 8, 60, 31, 4, 90, 55, 18, 68, 34, 29, 52, 28, 93, 43, 89, 39]


In [12]:
ten_cards = deck.deal_cards(10)
ten_cards

[22, 20, 39, 26, 49, 6, 98, 52, 51, 54]

In [14]:
rabbit.take_cards(ten_cards)
rabbit.show_info()

Name: Rabbit
Strategies: give_random, take_random
Cards: [22, 20, 39, 26, 49, 6, 98, 52, 51, 54]
Points: 0


In [15]:
rabbit.cards

[22, 20, 39, 26, 49, 6, 98, 52, 51, 54]

In [21]:
jenny.name

'Jenny'

In [9]:
deck = Deck()
deck.show_info(15)
x = deck.deal_cards(10)
deck.show_info(7)

Showing DECK info
Cards left: 104
Showing last 15 cards: [1, 92, 65, 80, 30, 68, 69, 67, 72, 29, 10, 32, 37, 95, 99]
Showing DECK info
Cards left: 94
Showing last 7 cards: [62, 104, 1, 92, 65, 80, 30]


In [10]:
x

[99, 95, 37, 32, 10, 29, 72, 67, 69, 68]

In [106]:
deck.cards.pop()
deck.show_info(3)

Showing DECK info
Cards left: 102
Showing last 3 cards: [64, 89, 66]


In [11]:
cards = list(range(1, 105))
print(cards)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]


In [23]:
me = {
    "name": 'Jenny',
    "age": 12,
    "school": 73,
    "city": 'Wroclaw',
}

In [25]:
me["name"] = "Tonia"
me["name"]

'Tonia'

In [26]:
me["year"] = 2011
me

{'name': 'Tonia', 'age': 12, 'school': 73, 'city': 'Wroclaw', 'year': 2011}

In [32]:
ochki = {}
for card in cards:
   
    if card == 55:
        points = 7
    elif card % 10 == 0:
        points = 3
    elif card % 5 == 0:
        points = 2
    elif card % 11 == 0:
        points = 5
    else:
        points = 1
    # print(f"card: {card}, points: {points}") 
    ochki[card] = points
print(ochki)
    

{1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 9: 1, 10: 3, 11: 5, 12: 1, 13: 1, 14: 1, 15: 2, 16: 1, 17: 1, 18: 1, 19: 1, 20: 3, 21: 1, 22: 5, 23: 1, 24: 1, 25: 2, 26: 1, 27: 1, 28: 1, 29: 1, 30: 3, 31: 1, 32: 1, 33: 5, 34: 1, 35: 2, 36: 1, 37: 1, 38: 1, 39: 1, 40: 3, 41: 1, 42: 1, 43: 1, 44: 5, 45: 2, 46: 1, 47: 1, 48: 1, 49: 1, 50: 3, 51: 1, 52: 1, 53: 1, 54: 1, 55: 7, 56: 1, 57: 1, 58: 1, 59: 1, 60: 3, 61: 1, 62: 1, 63: 1, 64: 1, 65: 2, 66: 5, 67: 1, 68: 1, 69: 1, 70: 3, 71: 1, 72: 1, 73: 1, 74: 1, 75: 2, 76: 1, 77: 5, 78: 1, 79: 1, 80: 3, 81: 1, 82: 1, 83: 1, 84: 1, 85: 2, 86: 1, 87: 1, 88: 5, 89: 1, 90: 3, 91: 1, 92: 1, 93: 1, 94: 1, 95: 2, 96: 1, 97: 1, 98: 1, 99: 5, 100: 3, 101: 1, 102: 1, 103: 1, 104: 1}


In [35]:
ochki[44] = 10
ochki[44]

10

In [37]:
ochki[200] = 3

In [40]:
l = [5, 1, 9]
l.sort()
l

[1, 5, 9]

In [41]:
min(l)

1

In [42]:
max(l)

9

In [43]:
class Student:
    def __init__(self, name, age, group, subjects):
        self.name = name
        self.age = age
        self.group = group
        self.subjects = subjects
        
    def show_info(self):
        info = "Student's info\n"
        info += f"Name: {self.name}\n"
        info += f"Age: {self.age}\n"
        info += f"Group: {self.group}\n"
        info += f"Subjects: {self.subjects}"
        print(info)

In [44]:
s1 = Student("Jenny", 12, "7a", ["Math", "English", "PE"])
print(s1.name)

Jenny


In [45]:
s2 = Student("Tonia", 16, "2b1", ["Polish", "English", "Biology"])
s2.subjects

['Polish', 'English', 'Biology']

In [46]:
s1.show_info()

Student's info
Name: Jenny
Age: 12
Group: 7a
Subjects: ['Math', 'English', 'PE']


In [47]:
s2.show_info()

Student's info
Name: Tonia
Age: 16
Group: 2b1
Subjects: ['Polish', 'English', 'Biology']


In [8]:
class Pet:
    def __init__(self, name, age, kind, color, weight):
        self.name = name
        self.age = age
        self.kind = kind
        self.color = color
        self.weight = weight
        
    def feed_me(self, food_kg):
        print(f"feeding {self.name}")
        print(f"{self.name} has eaten {food_kg} kg of food.")
        self.weight+=food_kg
        print(f"Now {self.name} weighs {self.weight} kg")

    def go_poo(self, mass):
        print(f"{self.name} is going to toilet")
        print(f"{self.name} vysrala {mass} kg")
        self.weight -= mass
        print(f"Now {self.name} weighs {self.weight} kg")

In [9]:
simka = Pet("Simka", 6, "cat", "grey", 4)
marusia = Pet("Marusia", 5, "dog", "orange", 10)


In [10]:
simka.feed_me(0.2)

feeding Simka
Simka has eaten 0.2 kg of food.
Now Simka weighs 4.2 kg


In [11]:
simka.weight

4.2

In [12]:
simka.feed_me(0.3)

feeding Simka
Simka has eaten 0.3 kg of food.
Now Simka weighs 4.5 kg


In [13]:
simka.weight

4.5

In [14]:
simka.go_poo(0.6)

Simka is going to toilet
Simka vysrala 0.6 kg
Now Simka weighs 3.9 kg


In [15]:
marusia.feed_me(2)

feeding Marusia
Marusia has eaten 2 kg of food.
Now Marusia weighs 12 kg


In [16]:
marusia.go_poo(1.8)

Marusia is going to toilet
Marusia vysrala 1.8 kg
Now Marusia weighs 10.2 kg


In [36]:
# data types 
# int 
x = 10
# float
x = 3.7
# str
x = "popa"
# bool
x = True
# list
x = [1, 2, 3.5, "popa", False, ["a", "b", True], 7]

In [71]:
# dict
x = {'name': 'Jenny',
     'age': 12,
     'subjects': ['math', 'infp', 'polish'],
     'married': True
    }

In [38]:
x['name']

'Jenny'

In [39]:
x['subjects']

['math', 'infp', 'polish']

In [40]:
x['subjects'][0]

'math'

In [41]:
x['country'] = 'blr'

In [42]:
x

{'name': 'Jenny',
 'age': 12,
 'subjects': ['math', 'infp', 'polish'],
 'married': True,
 'country': 'blr'}

In [43]:
x['married'] = False

In [70]:
x

[]

In [72]:
len(x['subjects'])

3

In [46]:
x

{'name': 'Jenny',
 'age': 12,
 'subjects': ['math', 'info', 'polish'],
 'married': False,
 'country': 'blr'}

In [77]:
x = [1,2,3,4,5]

In [84]:
x[-2:]

[4, 5]

In [85]:
x[-3:]

[3, 4, 5]

In [86]:
q = 2

In [89]:
x[-q:]

[4, 5]

# `for` loops

In [13]:
x = range(41)

In [14]:
list(x)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40]

In [19]:
for num in x:
    
    if num % 10 == 0:
        print(f'{num} ends with zero')
    elif num % 2 == 0:
        print(f'{num} is an even number')
    else:
        print(f'{num} is an odd number') 

0 ends with zero
1 is an odd number
2 is an even number
3 is an odd number
4 is an even number
5 is an odd number
6 is an even number
7 is an odd number
8 is an even number
9 is an odd number
10 ends with zero
11 is an odd number
12 is an even number
13 is an odd number
14 is an even number
15 is an odd number
16 is an even number
17 is an odd number
18 is an even number
19 is an odd number
20 ends with zero
21 is an odd number
22 is an even number
23 is an odd number
24 is an even number
25 is an odd number
26 is an even number
27 is an odd number
28 is an even number
29 is an odd number
30 ends with zero
31 is an odd number
32 is an even number
33 is an odd number
34 is an even number
35 is an odd number
36 is an even number
37 is an odd number
38 is an even number
39 is an odd number
40 ends with zero


In [20]:
d = {'name': 'Jenny',
     'age': 12,
     'subjects': ['math', 'infp', 'polish'],
     'married': True
    }

In [24]:
d.items()

dict_items([('name', 'Jenny'), ('age', 12), ('subjects', ['math', 'infp', 'polish']), ('married', True)])

In [25]:
t = (1,2,3)

In [26]:
t[0]

1

In [27]:
m= [1,2,3]

In [28]:
m[0] = 5

In [29]:
m

[5, 2, 3]

In [30]:
t[0] = 0

TypeError: 'tuple' object does not support item assignment

In [31]:
d.keys()

dict_keys(['name', 'age', 'subjects', 'married'])

In [61]:
letter = "m"
t = 0
for key in d.keys():
    l = len(key)
    if letter in key:
        print(f'The word "{key}" has letter "{letter}"')
        f = key.index(letter)
        print(f'\tThe position of the letter "{letter}" is {f+1}')
    else:
        print(f'The word "{key}" does not have letter "{letter}"')
    print(f'\t\tThere are {l} letters in the word "{key}"\n')
    t+=l
print(f'There are {t} letters in all words')

The word "name" has letter "m"
	The position of the letter "m" is 3
		There are 4 letters in the word "name"

The word "age" does not have letter "m"
		There are 3 letters in the word "age"

The word "subjects" does not have letter "m"
		There are 8 letters in the word "subjects"

The word "married" has letter "m"
	The position of the letter "m" is 1
		There are 7 letters in the word "married"

There are 22 letters in all words


In [32]:
name = 'Jenny'

In [33]:
"n" in name

True

In [1]:
class Chel:
    def __init__(self, name, age, money, grades):
        self.name = name
        self.age = age
        self.money = money
        self.grades = grades
    
    def show_info(self):
        print(self.__dict__)
        
    def get_grade(self, new_grade):
        self.grades.append(new_grade)
        
    def show_grades(self):
        print(f"{self.name}'s grades are: {self.grades}")
        
    def get_dengi(self, kwota):
        if kwota <= 0:
            print(f"{self.name} said 'A gde dengi?'")
            return
        else:
            self.money += kwota
        
        if kwota <= 100:
            print(f"{self.name} said 'spasibo'")
        else:
            print(f"{self.name} said 'spasiba, bro'")
        
    def show_dengi(self):
        print(f"{self.name} has {self.money} deneg")
        
    def give_dengi(self, kwota):
        if kwota <= 0:
            print("Cho")
            return 0
        elif kwota > self.money:
            print(f'''{self.name} said "Sorry, bro"''')
            return 0
        else:
            self.money -= kwota
            return kwota
    
    def birth_year(self):
        return 2023 - self.age

In [2]:
jenny = Chel("Jenny", 12, 200, [5,5,4,5])
roma = Chel('Roma', 18, 300, None)
tonia = Chel('Tonia', 16 , 200, [5,6,3,4])

In [3]:
jenny.show_info()
roma.show_info()
tonia.show_info()

{'name': 'Jenny', 'age': 12, 'money': 200, 'grades': [5, 5, 4, 5]}
{'name': 'Roma', 'age': 18, 'money': 300, 'grades': None}
{'name': 'Tonia', 'age': 16, 'money': 200, 'grades': [5, 6, 3, 4]}


In [101]:
jenny.get_dengi(roma.give_dengi(50))


Jenny said 'spasibo'


In [103]:
jenny.get_dengi(tonia.give_dengi(10))

Jenny said 'spasibo'


In [4]:
by = jenny.birth_year()

In [5]:
by

2011

In [44]:
import pandas as pd

In [50]:
result = {
    "p0": [32, 10, 50, 67, 4, None],
    "p1": [23, 54, 73, 3, -5, False],
    "p2": [45, 72, 34, 0, True, 4],
    "p3": [56, 23, 8 ,6, "pon", 432]
}

In [51]:
df = pd.DataFrame(result)
df

,p0,p1,p2,p3
0,32.0,23,45,56
1,10.0,54,72,23
2,50.0,73,34,8
3,67.0,3,0,6
4,4.0,-5,True,pon
5,NaN,False,4,432
